# Version optimized
Phiên bản này được tối ưu hóa bằng cách sử dụng batching và cải thiện hiệu suất xử lý.

In [ ]:
# Loguru tự động ghi lại traceback (dấu vết lỗi) mà không cần cấu hình thêm
# ! pip install loguru

In [7]:
# ! pip install opencv-python opencv-contrib-python

   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.3/46.2 MB ? eta -:--:--
   - -------------------------------------- 1.6/46.2 MB 7.0 MB/s eta 0:00:07
   -- ------------------------------------- 3.1/46.2 MB 6.4 MB/s eta 0:00:07
   ---- ----------------------------------- 4.7/46.2 MB 6.8 MB/s eta 0:00:07
   ----- ---------------------------------- 6.8/46.2 MB 7.6 MB/s eta 0:00:06
   ------- -------------------------------- 9.2/46.2 MB 8.2 MB/s eta 0:00:05
   --------- ------------------------------ 11.3/46.2 MB 8.5 MB/s eta 0:00:05
   ----------- ---------------------------- 13.6/46.2 MB 8.9 MB/s eta 0:00:04
   ------------- -------------------------- 15.7/46.2 MB 9.1 MB/s eta 0:00:04
   --------------- ------------------------ 18.1/46.2 MB 9.4 MB/s eta 0:00:04
   ----------------- ---------------------- 20.4/46.2 MB 9.5 MB/s eta 0:00:03
   ----------------- ---------------------- 20.4/46.2 MB 9.5 MB/s eta 0:00:03
   ----

In [5]:
import argparse
from collections import defaultdict
import cv2
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
from loguru import logger

## Định nghĩa cấu hình và khởi tạo video

In [ ]:
def load_config():
    return {
        "model_path": "yolo11l.pt",
        "track_history_length": 120,
        "batch_size": 64,
        "line_thickness": 4,
        "track_color": [230, 230, 230]
    }

def initialize_video(video_path):
    cap = cv2.VideoCapture(video_path)
    # get property
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH | 3))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT | 4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    size = (width, height)
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    file_name = video_path.split("/")[-1].split(".")[0]
    output_path = f"./data/output/{file_name}_tracking.mp4"

    out = cv2.VideoWriter(output_path, fourcc, fps, size)

    return cap, out, output_path

## Cập nhật lịch sử theo dõi: Cập nhật lịch sử theo dõi và loại bỏ các track cũ.

In [ ]:
def update_track_history(track_history, last_seen, track_ids, frame_count, batch_size, frame_idx, history_length):
    """Update tracking history and remove old tracks."""
    current_tracks = set(track_ids)
    for track_id in list(track_history.keys()):
        if track_id in current_tracks:
            # Cập nhật thời điểm cuối cùng đối tượng được nhìn thấy
            last_seen[track_id] = frame_count - (batch_size - frame_idx - 1)
        elif frame_count - last_seen[track_id] > history_length:
            # Xóa đối tượng nếu đã quá lâu không xuất hiện
            del track_history[track_id]
            del last_seen[track_id]

## Vẽ các đường theo dõi trên frame

In [ ]:
def draw_tracks(boxes, track_ids, track_history, annotated_frame, config):
    for box, track_id in zip(boxes, track_ids):
        x, y, w, h = box
        track = track_history[track_id]
        track.append((float(x), float(y)))  # x, y center point
        # retain 120 tracks for 120 frames
        if len(track) > config['track_history_length']:
            track.pop(0)

            # Draw the tracking lines
        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(annotated_frame, [points], isClosed=False, color=
            config['track_color'], thickness=config['line_thickness'])
        
        return annotated_frame

## Xử lý một batch các frames

In [ ]:
def process_batch (model, batch_frames, track_history, last_seen, frame_count, config):
    """Process a batch of frames through YOLO model"""
    results = model.track(
        batch_frames,
        persist=True,
        tracker="botsort.yaml",
        show=False,
        verbose=False,
        iou=0.5,
    )
    return results

## Main Processing video

In [ ]:
def main(video_path):
    config = load_config()
    model = YOLO(config['model_path'])
    cap, out, output_path = initialize_video(video_path)
    track_history = defaultdict(lambda : list)
    last_seen = defaultdict(int)

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))